<a href="https://colab.research.google.com/github/ravi-prakash1907/A-tracking-of-COVID-19/blob/master/frombeg/smsClassifactio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prepare the dataset

#### Tokanize and all


In [24]:
## libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import requests
import re

In [2]:
## location to store the dataset
try:
  os.mkdir('Datasets')
  dataFile = 'Datasets/SMSSpamCollection.txt'
except:
  dataFile = 'Datasets/SMSSpamCollection.txt'

In [3]:
## fun. to download the data from any url
def downloadData(fileURL, saveAs='downloaded'):
  req = requests.get(fileURL)
  fileURLContent = req.content
  data = open(saveAs, 'wb')
  
  data.write(fileURLContent)
  data.close()

In [4]:
## location of the file
fileURL = 'https://raw.githubusercontent.com/ravi-prakash1907/Machine-Learning-for-Cyber-Security/main/Datasets/smsspamcollection/SMSSpamCollection.txt?token=AJGAAOA2SOYVM324GI45BLTATFASY'

## downloading the from url
downloadData(fileURL,dataFile)

In [5]:
## importing data
sms_spam = pd.read_csv(dataFile, sep='\t', header=None, names=['Label', 'SMS'])

print("Shape",sms_spam.shape)
sms_spam.head()

Shape (5572, 2)


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
## checking for ham/spam SMS ratio from the whole bulk
sms_spam['Label'].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

## Data Cleaning

In [7]:
### cleaning
sms_spam['SMS'] = sms_spam['SMS'].str.lower()
sms_spam.head(3)

,Label,SMS
0,ham,"go until jurong point, crazy.. available only ..."
1,ham,ok lar... joking wif u oni...
2,spam,free entry in 2 a wkly comp to win fa cup fina...


In [7]:
## removing the blank spaces
sms_spam['SMS'] = sms_spam['SMS'].str.strip()
sms_spam.head(3)

In [8]:
## removing punctuations
def removePunctuations(gotStr):
  cleanedStr = re.sub(r'[^\w\s]', '', gotStr)
  return cleanedStr

sms_spam['SMS'] = sms_spam['SMS'].apply(removePunctuations)
sms_spam.head()

,Label,SMS
0,ham,go until jurong point crazy available only in ...
1,ham,ok lar joking wif u oni
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor u c already then say
4,ham,nah i dont think he goes to usf he lives aroun...


### Data Preparation

In [14]:
numLab = [int(s == 'spam') for s in sms_spam['Label']]
sms_spam['Num Label'] = numLab
sms_spam.head()

,Label,SMS,Num Label
0,ham,go until jurong point crazy available only in ...,0
1,ham,ok lar joking wif u oni,0
2,spam,free entry in 2 a wkly comp to win fa cup fina...,1
3,ham,u dun say so early hor u c already then say,0
4,ham,nah i dont think he goes to usf he lives aroun...,0


In [21]:
## changing to lower case
sms_spam['SMS'] = sms_spam['SMS'].str.split()

vocabulary = []
for sms in sms_spam['SMS']:
   for word in sms:
      vocabulary.append(word)

vocabulary = list(set(vocabulary))

In [22]:
word_counts_per_sms = {unique_word: [0] * len(sms_spam['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(sms_spam['SMS']):
   for word in sms:
      word_counts_per_sms[word][index] += 1

In [19]:
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,bak,inforingtonekingcouk,fringe,progress,honeymoon,liverpool,js,intentions,relaxing,sirs,forwarded,buttheres,08718729755,09061702893,youd,causes,these,prayingwill,finished,timin,pobox84,meanwhile,www80488biz,complexities,lucozade,meetitz,typical,youuuuu,dl,dismay,library,raping,soil,1500,consensus,10803,wrecked,guessing,dearer,units,...,gin,hopeso,africa,partner,5gardener,hmv,insha,tulip,omw,complete,zeros,imposed,mattermsg,interviw,felt,645,overemphasiseor,applying,giv,mail,aunt,act,photoshop,opponenter,nange,69200,swimsuit,ree,mornin,teach,0,morrow,redeemable,exactly,woozles,digital,calls1minmobsmore,contacted,processits,noun
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [20]:
word_counts['Label'] = sms_spam['Num Label']
word_counts.head()

,bak,inforingtonekingcouk,fringe,progress,honeymoon,liverpool,js,intentions,relaxing,sirs,forwarded,buttheres,08718729755,09061702893,youd,causes,these,prayingwill,finished,timin,pobox84,meanwhile,www80488biz,complexities,lucozade,meetitz,typical,youuuuu,dl,dismay,library,raping,soil,1500,consensus,10803,wrecked,guessing,dearer,units,...,hopeso,africa,partner,5gardener,hmv,insha,tulip,omw,complete,zeros,imposed,mattermsg,interviw,felt,645,overemphasiseor,applying,giv,mail,aunt,act,photoshop,opponenter,nange,69200,swimsuit,ree,mornin,teach,0,morrow,redeemable,exactly,woozles,digital,calls1minmobsmore,contacted,processits,noun,Label
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


---  
---

## Naive Bayes

In [23]:
df = word_counts.copy()
df.head()

,bak,inforingtonekingcouk,fringe,progress,honeymoon,liverpool,js,intentions,relaxing,sirs,forwarded,buttheres,08718729755,09061702893,youd,causes,these,prayingwill,finished,timin,pobox84,meanwhile,www80488biz,complexities,lucozade,meetitz,typical,youuuuu,dl,dismay,library,raping,soil,1500,consensus,10803,wrecked,guessing,dearer,units,...,hopeso,africa,partner,5gardener,hmv,insha,tulip,omw,complete,zeros,imposed,mattermsg,interviw,felt,645,overemphasiseor,applying,giv,mail,aunt,act,photoshop,opponenter,nange,69200,swimsuit,ree,mornin,teach,0,morrow,redeemable,exactly,woozles,digital,calls1minmobsmore,contacted,processits,noun,Label
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
df.describe()

,bak,inforingtonekingcouk,fringe,progress,honeymoon,liverpool,js,intentions,relaxing,sirs,forwarded,buttheres,08718729755,09061702893,youd,causes,these,prayingwill,finished,timin,pobox84,meanwhile,www80488biz,complexities,lucozade,meetitz,typical,youuuuu,dl,dismay,library,raping,soil,1500,consensus,10803,wrecked,guessing,dearer,units,...,hopeso,africa,partner,5gardener,hmv,insha,tulip,omw,complete,zeros,imposed,mattermsg,interviw,felt,645,overemphasiseor,applying,giv,mail,aunt,act,photoshop,opponenter,nange,69200,swimsuit,ree,mornin,teach,0,morrow,redeemable,exactly,woozles,digital,calls1minmobsmore,contacted,processits,noun,Label
count,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.00000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,...,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000
mean,0.001436,0.000179,0.000359,0.000179,0.000179,0.000538,0.000179,0.000179,0.000179,0.000179,0.001615,0.000179,0.000179,0.000359,0.001436,0.000179,0.003948,0.000179,0.00341,0.000179,0.001077,0.000718,0.000179,0.000179,0.000359,0.000179,0.000179,0.000179,0.000179,0.000179,0.001436,0.000179,0.000179,0.000718,0.000179,0.000179,0.000179,0.000359,0.000538,0.000179,...,0.000179,0.000179,0.000897,0.000538,0.001795,0.000179,0.000359,0.001256,0.001436,0.000179,0.000179,0.000179,0.000179,0.002333,0.000179,0.000179,0.000179,0.000538,0.004307,0.000359,0.000179,0.000179,0.000179,0.000179,0.000179,0.000179,0.000179,0.000359,0.001077,0.000538,0.000179,0.000179,0.001077,0.000179,0.001436,0.000179,0.000897,0.000179,0.000359,0.134063
std,0.037867,0.013397,0.018944,0.013397,0.013397,0.023199,0.013397,0.013397,0.013397,0.013397,0.040161,0.013397,0.013397,0.018944,0.037867,0.013397,0.062717,0.013397,0.05830,0.013397,0.032800,0.026786,0.013397,0.013397,0.018944,0.013397,0.013397,0.013397,0.013397,0.013397,0.037867,0.013397,0.013397,0.026786,0.013397,0.013397,0.013397,0.018944,0.023199,0.013397,...,0.013397,0.013397,0.029945,0.023199,0.070872,0.013397,0.018944,0.035425,0.037867,0.013397,0.013397,0.013397,0.013397,0.048250,0.013397,0.013397,0.013397,0.023199,0.068180,0.018944,0.013397,0.013397,0.013397,0.013397,0.013397,0.013397,0.013397,0.018944,0.032800,0.023199,0.013397,0.013397,0.032800,0.013397,0.037867,0.013397,0.029945,0.013397,0.018944,0.340751
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,

### Data Praparation

In [26]:
def groupCol(dataCol):
  #return [int(np.isnan(val)^bool(val)) for val in dataCol//10]
  dataCol = pd.Series(dataCol)
  return [str(val) for val in dataCol//10]

In [27]:
def prepare(df):
  ## grouping all cols
  for col in tuple(df.columns[1:-1]):
    df[col] = groupCol(dataCol = df[col])

### Naive Bayes' Classifier
_It takes the tuples for set of the:_  
1. Dependent variables  
2. Values of any datapoint
3. Name of cloumn i.e. to be predicted

In [28]:
## gives rows with given set of cols
def getOneLabel(df,label,predLabel):
  tempDf = df.copy()
  x = [df.loc[i][label] in predLabel for i in df.index]
  count = 0
  ind = []
  for val in x:
    if not val:
      ind.append(count)
    count += 1
  
  #print(ind)
  tempDf.drop(ind, inplace=True)
  prepare(tempDf)
  return tempDf

In [29]:
## classifier
def naiveBayesianPredictor(df, labels, given, predCol):
  possibleCases = list(df[predCol].unique())
  finalPredDescription = {}
  
  for possibleCase in possibleCases:
    tempDF = getOneLabel(df, predCol, [possibleCase])
    preds = len(tempDF) / len(df)


    for index in range(len(given)):
      partialProb = tempDF[labels[index]].value_counts() / len(tempDF)

      if given[index] not in np.array(partialProb.index, dtype='uint64'):    
        tempList = list(np.array(partialProb.index, dtype='uint64'))
        largers = [val for val in tempList if val>int(given[index])]
        if largers and int(given[index])/largers[0] >= 0.5:
          preds *= partialProb[str(largers[0])]
        else:
          continue
      else:
        preds *= partialProb[given[index]]  #Naive bayesian classifier
    
    finalPredDescription[possibleCase] = preds
  
  ## converting predictions to pandas Series
  finalPredDescription = pd.Series(finalPredDescription)
  ## choosing MAP (Maximum A Posteriori)
  if finalPredDescription[0] == finalPredDescription[1]:
    finalPred = -1
  else:
    finalPred = str(finalPredDescription[finalPredDescription == max(finalPredDescription)].index[0])

  return finalPred,finalPredDescription

In [30]:
def predict(df,labels, given, predCol, describe = False):
  given = groupCol(dataCol=given)
  
  finalPrediction = naiveBayesianPredictor(df, labels, given, predVar)
  
  if finalPrediction == -1:
    print("Tie for the given e-mail!! Get a human to check for spam!")
  else:
    res = ' ' if bool(int(finalPrediction[0])) else ' NOT '
    print("\nGiven e-mail is{}Spam! \nLabel Value: {}".format(res,finalPrediction[0]))
  if describe:
    print("\nHere 1 indicates the probability of being a Spam E-Mail!")
    print("Probability Table:")
    return pd.DataFrame([finalPrediction[1]])


### Train-Test Split

In [33]:
# Randomize the dataset
data_randomized = df.sample(frac=1, random_state=1)

# Calculate index for split
training_test_index = round(len(data_randomized) * 0.8)

# Split into training and test sets
training_set = data_randomized[:training_test_index].reset_index(drop=True)
test_set = data_randomized[training_test_index:].reset_index(drop=True)

print(training_set.shape)
print(test_set.shape)

(4458, 9586)
(1114, 9586)


In [34]:
training_set['Label'].value_counts(normalize=True)

0    0.86541
1    0.13459
Name: Label, dtype: float64

In [35]:
test_set['Label'].value_counts(normalize=True)

0    0.868043
1    0.131957
Name: Label, dtype: float64

In [36]:
# After splitting
training_set.head(3)

,bak,inforingtonekingcouk,fringe,progress,honeymoon,liverpool,js,intentions,relaxing,sirs,forwarded,buttheres,08718729755,09061702893,youd,causes,these,prayingwill,finished,timin,pobox84,meanwhile,www80488biz,complexities,lucozade,meetitz,typical,youuuuu,dl,dismay,library,raping,soil,1500,consensus,10803,wrecked,guessing,dearer,units,...,hopeso,africa,partner,5gardener,hmv,insha,tulip,omw,complete,zeros,imposed,mattermsg,interviw,felt,645,overemphasiseor,applying,giv,mail,aunt,act,photoshop,opponenter,nange,69200,swimsuit,ree,mornin,teach,0,morrow,redeemable,exactly,woozles,digital,calls1minmobsmore,contacted,processits,noun,Label
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


---  
---

In [31]:
labels = tuple(df.columns[1:-1])
predVar = df.columns[-1]